In [ ]:
import streamlit as st
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Sample FAQ database
faqs = {
    "What is AI?": "Artificial Intelligence (AI) is the simulation of human intelligence in machines that can perform tasks like learning and problem-solving.",
    "How does machine learning work?": "Machine learning is a subset of AI that enables systems to learn from data patterns without explicit programming.",
    "What is deep learning?": "Deep learning is a branch of machine learning using neural networks with multiple layers to analyze complex data.",
}

# Convert FAQ questions into embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()

faq_embeddings = np.array([get_embedding(q) for q in faqs.keys()])
index = faiss.IndexFlatL2(faq_embeddings.shape[2])
index.add(faq_embeddings.reshape(len(faqs), -1))

# Streamlit UI
st.title("Smart FAQ Assistant")
user_query = st.text_input("Ask a question:")

if user_query:
    query_embedding = get_embedding(user_query).reshape(1, -1)
    _, idx = index.search(query_embedding, 1)
    best_match = list(faqs.keys())[idx[0][0]]
    st.write("**Answer:**", faqs[best_match])
